
# PyTorch Quickstart

Have a pytorch model? This notebook will show you how to get it set up in Lens. If this is your first time using Lens, we suggest starting with our [quickstart notebook](https://credoai-lens.readthedocs.io/en/stable/notebooks/quickstart.html) which takes you through the basics with an sklearn model.

**Setup**

Lens installation instruction can be found on [readthedocs](https://credoai-lens.readthedocs.io/en/stable/pages/setup.html)

**Find the code**

### Get your pytorch model and dataloaders ready.
 In this tutorial we will emulate the modeling phase by running a quick script. This script following the [MNIST quickstart](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html) from the pytorch documentation. You can see the full script [here](https://raw.githubusercontent.com/credo-ai/credoai_lens/main/credoai/demo_assets/_fetch_pytorch_mnist_model.py).


Because one of the primary use cases for Lens is to easily add Fairness and bias assessment to your pipeline, we are going to make up a sensitive features array
for the purpose of demonstration.

In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
# model and X_test, y_test, etc. are defined by this script
from credoai.demo_assets import fetch_mnist_model

# model and data are defined by this script
model, train_dataloader, test_dataloader = fetch_mnist_model()

# making up some sensitive features
import numpy as np
import pandas as pd
sensitive_categories = ['black', 'white']
sensitive_features = pd.Series(np.random.choice(sensitive_categories, size=len(test_dataloader.dataset)),
                                name='color')

### Lens

Below is a basic example where our goal is to evaluate the above model.

Briefly, the code is doing four things:

* Wrapping ML artifacts (like models and data) in Lens objects
* Initializing an instance of Lens. Lens is the main object that performs evaluations. Under the hood, it creates a `pipeline` of evaluations that are run.
* Add evaluators to Lens.
* Run Lens

In [ ]:
# Import Lens and necessary artifacts
from credoai.lens import Lens
from credoai.artifacts import ClassificationModel, PytorchData

In lens, the classes that evaluate models and/or datasets are called `evaluators`. In this example we are interested in evaluating the model's fairness. For this we can use the `ModelFairness` evaluator. We'll
also evaluate the model's performance.

In [ ]:
from credoai.evaluators import ModelFairness, Performance

## Lens in 5 minutes

Below is a basic example where our goal is to evaluate the above model. We'll break down this code [below](#Breaking-Down-The-Steps).

Briefly, the code is doing four things:

* Wrapping ML artifacts (like models and data) in Lens objects
* Initializing an instance of Lens. Lens is the main object that performs evaluations. Under the hood, it creates a `pipeline` of evaluations that are run.
* Add evaluators to Lens.
* Run Lens

In [ ]:
credo_data = PytorchData(
    name = 'mnist-data',
    dataloader=test_dataloader,
    sensitive_features=sensitive_features
)

credo_model = ClassificationModel(
    name='mnist-classifier',
    model_like=model,
    tags=None
)

# Initialization of the Lens object
lens = Lens(model=credo_model, assessment_data=credo_data)

# initialize the evaluator and add it to Lens
metrics = ['precision_score', 'recall_score', 'equal_opportunity']
lens.add(ModelFairness(metrics=metrics))
lens.add(Performance(metrics=metrics))

# run Lens
lens.run()